In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [11]:
import os
import functools as ft
files = os.listdir('../dataset')
dfs=[]
for file in files:
    dftmp = pd.read_csv('../dataset'+'/'+file)
    dftmp = dftmp[['date','time','REPORT_ID','DISTANCE_IN_METERS','vehicleCount']]
    dftmp['time_full'] = dftmp['time'].astype(str)
    dftmp['time_full'] = dftmp['time_full'].apply(lambda x: str(x) if (len(x)==2) else '0'+x) 
    dftmp['time_full'] = dftmp['time_full'].apply(lambda x: x+':00:00')
    dftmp['timestamp'] = dftmp['date']+' '+dftmp['time_full']
    dftmp.drop(['date','time_full'], axis=1, inplace=True)
    dftmp['day_of_week'] = pd.to_datetime(dftmp['timestamp']).dt.dayofweek
    dfs.append(dftmp)
df = pd.concat(dfs)

In [12]:
#create lag data
lag_df = df.copy()
for i in range(1,3):
    lag_df['vehicleCount_lag_'+str(i)] = lag_df.groupby(['REPORT_ID'])['vehicleCount'].shift(i)
cols = ['time','day_of_week','REPORT_ID','DISTANCE_IN_METERS','vehicleCount_lag_1','vehicleCount_lag_2','vehicleCount']
lag_df = lag_df[cols]
lag_df = lag_df.dropna()
# lag_df = lag_df.reset_index(drop=True)

In [13]:
lag_df.head(3)

,time,day_of_week,REPORT_ID,DISTANCE_IN_METERS,vehicleCount_lag_1,vehicleCount_lag_2,vehicleCount
2,2,1,158895,1505,1.0,1.0,19
3,3,1,158895,1505,19.0,1.0,28
4,4,1,158895,1505,28.0,19.0,126


In [14]:
lag_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 642281 entries, 2 to 10775
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   time                642281 non-null  int64  
 1   day_of_week         642281 non-null  int64  
 2   REPORT_ID           642281 non-null  int64  
 3   DISTANCE_IN_METERS  642281 non-null  int64  
 4   vehicleCount_lag_1  642281 non-null  float64
 5   vehicleCount_lag_2  642281 non-null  float64
 6   vehicleCount        642281 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 39.2 MB


In [15]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [17]:
class MyDecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        print('X: ', X)
        print('Y: ', Y)
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [18]:
X = lag_df.iloc[:, :-1].values
Y = lag_df.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [19]:
regressor = MyDecisionTreeRegressor(min_samples_split=3, max_depth=3)
regressor.fit(X_train,Y_train)

X:  [[9.00000e+00 5.00000e+00 1.94878e+05 5.57000e+02 1.47000e+02 8.10000e+01]
 [1.90000e+01 5.00000e+00 2.03769e+05 5.74000e+02 3.00000e+00 7.00000e+00]
 [1.00000e+01 6.00000e+00 2.04300e+05 5.45000e+02 0.00000e+00 0.00000e+00]
 ...
 [1.10000e+01 4.00000e+00 1.87695e+05 7.52000e+02 4.60000e+01 6.90000e+01]
 [1.70000e+01 4.00000e+00 1.87960e+05 7.37000e+02 6.00000e+00 9.00000e+00]
 [8.00000e+00 5.00000e+00 1.85422e+05 7.05000e+02 1.70000e+01 1.40000e+01]]
Y:  [[183]
 [  0]
 [  4]
 ...
 [ 51]
 [  0]
 [  7]]


In [20]:
Y_pred = regressor.predict(X_test) 
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(Y_test, Y_pred))

24.571939221004097

In [21]:
from sklearn.metrics import r2_score    
np.sqrt(r2_score(Y_test, Y_pred))

0.9171163708897956

In [22]:
#save the model to disk
import pickle
filename = 'MyDecisionTreeRegressorModelWithDOW.sav'
pickle.dump(regressor, open(filename, 'wb'))